In [ ]:
# Install required libraries
!pip install pdfplumber
!pip install sentence-transformers
!pip install faiss-cpu
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 51.1 MB/s eta 0:00:00


In [ ]:
import pdfplumber
from google.colab import files

# Upload the PDF file
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

# Extract content from the PDF
with pdfplumber.open(pdf_path) as pdf:
    # Page 2: Extract unemployment information
    page2_text = pdf.pages[1].extract_text()
    # Page 6: Extract table data
    page6_table = pdf.pages[5].extract_table()

print("Page 2 Text:\n", page2_text)
print("\nPage 6 Table:\n", page6_table)

Saving Tabless.pdf to Tabless.pdf
Page 2 Text:
 

Page 6 Table:
 [['Year', '2010', '2011', '2012', '2013', '2014', '2015'], ['All Industries', '26093515', '27535971', '28663246', '29601191', '30895407', '31397023'], ['Manufacturing', '4992521', '5581942', '5841608', '5953299', '6047477', '5829554'], ['Finance,\nInsurance, Real\nEstate, Rental,\nLeasing', '4522451', '4618678', '4797313', '5031881', '5339678', '5597018'], ['Arts,\nEntertainment,\nRecreation,\nAccommodation,\nand Food Service', '964032', '1015238', '1076249', '1120496', '1189646', '1283813'], ['Other', '15614511', '16320113', '16948076', '17495515', '18318606', '18686638']]


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Initialize the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Prepare chunks for embedding
chunks = [page2_text] + [str(row) for row in page6_table]  # Combine text and table rows
embeddings = model.encode(chunks)  # Generate embeddings

# Store embeddings in FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)  # L2 similarity index
index.add(np.array(embeddings))

print("Embeddings stored in FAISS index.")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings stored in FAISS index.


In [ ]:
def search(query, model, index, chunks):
    # Generate query embedding
    query_embedding = model.encode([query])

    # Search for the top-3 most similar chunks
    _, indices = index.search(np.array(query_embedding), k=3)
    results = [chunks[i] for i in indices[0]]
    return results

# Example: User inputs a query
user_query = input("Enter your query: ")
retrieved_chunks = search(user_query, model, index, chunks)

print("Retrieved Chunks:\n", retrieved_chunks)

Enter your query: Show the data from the table on page 6
Retrieved Chunks:
 ["['Manufacturing', '4992521', '5581942', '5841608', '5953299', '6047477', '5829554']", "['Other', '15614511', '16320113', '16948076', '17495515', '18318606', '18686638']", "['Finance,\\nInsurance, Real\\nEstate, Rental,\\nLeasing', '4522451', '4618678', '4797313', '5031881', '5339678', '5597018']"]


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the local LLM model and tokenizer
model_name = "google/flan-t5-base"  # You can try "google/flan-t5-large" for better responses
tokenizer = AutoTokenizer.from_pretrained(model_name)
llm = AutoModelForSeq2SeqLM.from_pretrained(model_name)

print("Local LLM model loaded successfully!")

# Generate a response using the retrieved chunks and user query
def generate_response_local(results, query):
    # Combine retrieved chunks into a context
    context = "\n".join(results)
    prompt = (
        f"Given the following retrieved information:\n{context}\n\n"
        f"Answer the question: {query}"
    )

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)

    # Generate a response
    outputs = llm.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Generate the response
final_response = generate_response_local(retrieved_chunks, user_query)
print("\nGenerated Response:\n", final_response)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Local LLM model loaded successfully!

Generated Response:
 ['[TABLECONTEXT]', '[TITLE]', 'United States Census Bureau'], ['[TABLECONTEXT]', '[TITLE]', 'United States Census Bureau'], ['[TABLECONTEXT]', '[TITLE]', 'United States Census Bureau'], ['[TABLECONTEXT]', '[TITLE]', 'United States Census Bureau'], ['[TABLECONTEXT]', '[TITLE]', 'Uni
